# Modelling CNN

## Import Library

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

from sklearn import model_selection
from sklearn.feature_extraction.text import TfidfVectorizer

import tensorflow as tf
from tensorflow.keras.layers import Input, Flatten, Dense, Conv1D, MaxPool1D, Dropout
# from tqdm import tqdm

import numpy as np
import sklearn.metrics
from sklearn.metrics import classification_report
from sklearn.metrics import precision_score, accuracy_score, recall_score, f1_score

import warnings
warnings.filterwarnings('ignore')
import random

np.random.seed(0)
tf.random.set_seed(0)
random.seed(0)

## Function for Average

In [10]:
def Average(lst):
    return sum(lst) / len(lst)

In [11]:
!pip install gdown

In [12]:
!gdown --id 1s-4XNKLxOiHe_uORpbhwF7pVZjCXME0e


/opt/conda/lib/python3.10/site-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1s-4XNKLxOiHe_uORpbhwF7pVZjCXME0e
To: /kaggle/working/dataset_50k.csv
100%|██████████████████████████████████████| 6.77M/6.77M [00:00<00:00, 22.8MB/s]


In [14]:
!gdown https://drive.google.com/uc?id=1fy2o5J5wFEd-ksO6fG9_XVQFssiS-nC4

Downloading...
From: https://drive.google.com/uc?id=1fy2o5J5wFEd-ksO6fG9_XVQFssiS-nC4
To: /kaggle/working/corpus_indonews_twitter.fasttext
100%|██████████████████████████████████████| 12.7M/12.7M [00:00<00:00, 35.2MB/s]


In [15]:
!gdown https://drive.google.com/uc?id=1pRsOh_uJgRh0aUKd3Et23oLM9CJJNYcl


Downloading...
From (original): https://drive.google.com/uc?id=1pRsOh_uJgRh0aUKd3Et23oLM9CJJNYcl
From (redirected): https://drive.google.com/uc?id=1pRsOh_uJgRh0aUKd3Et23oLM9CJJNYcl&confirm=t&uuid=203ae421-37dc-4c04-94d3-af82128e7271
To: /kaggle/working/corpus_indonews_twitter.fasttext.wv.vectors_ngrams.npy
100%|█████████████████████████████████████████| 800M/800M [00:07<00:00, 108MB/s]


## Import Dataset

In [18]:
df = pd.read_csv('dataset_50k.csv', usecols = ['detokenize', 'Label'])


In [19]:
df = df.dropna()
df

Label                                         detokenize
0          1  aku luka diri aku jalan hidup sulit orang tua ...
1          0   buang pikir negatif laku gelisah milik putus asa
2          1  jujur aku takut mati aku orang aku sulit paham...
3          0              saya mes pasang sepatu jam saya harap
4          1  capek hati banget aku allah aku kayak mata ora...
...      ...                                                ...
50518      0  curi rampok liar begal rakyat senantiasa resah...
50519      0                      saya bicara saya buat tertawa
50520      1  sakit sakit guna sakit bahagia sakit bohong or...
50521      1                              lantai lantai gelisah
50522      1  aku tulis catat kemarin kata tunang aku laku b...

[50523 rows x 2 columns]

In [21]:
x = df['detokenize'].copy()
y = df['Label'].copy()

## Feature Extraction (TF-IDF)

In [22]:
x

0        aku luka diri aku jalan hidup sulit orang tua ...
1         buang pikir negatif laku gelisah milik putus asa
2        jujur aku takut mati aku orang aku sulit paham...
3                    saya mes pasang sepatu jam saya harap
4        capek hati banget aku allah aku kayak mata ora...
                               ...                        
50518    curi rampok liar begal rakyat senantiasa resah...
50519                        saya bicara saya buat tertawa
50520    sakit sakit guna sakit bahagia sakit bohong or...
50521                                lantai lantai gelisah
50522    aku tulis catat kemarin kata tunang aku laku b...
Name: detokenize, Length: 50523, dtype: object

In [23]:
%%time
# define tf-idf
tf_idf_vec = TfidfVectorizer(use_idf=True, smooth_idf=False,
                             ngram_range=(1,1), max_features=10000)

#create dataframe
x_uni = pd.DataFrame(tf_idf_vec.fit_transform(x).toarray(), columns=tf_idf_vec.get_feature_names_out())
y_uni = y

x_uni

CPU times: user 1.44 s, sys: 2.03 s, total: 3.47 s
Wall time: 3.49 s


aba  abad  abadi  abah  abai  abang  abar  abdi  abet  abis  ...  zero  \
0      0.0   0.0    0.0   0.0   0.0    0.0   0.0   0.0   0.0   0.0  ...   0.0   
1      0.0   0.0    0.0   0.0   0.0    0.0   0.0   0.0   0.0   0.0  ...   0.0   
2      0.0   0.0    0.0   0.0   0.0    0.0   0.0   0.0   0.0   0.0  ...   0.0   
3      0.0   0.0    0.0   0.0   0.0    0.0   0.0   0.0   0.0   0.0  ...   0.0   
4      0.0   0.0    0.0   0.0   0.0    0.0   0.0   0.0   0.0   0.0  ...   0.0   
...    ...   ...    ...   ...   ...    ...   ...   ...   ...   ...  ...   ...   
50518  0.0   0.0    0.0   0.0   0.0    0.0   0.0   0.0   0.0   0.0  ...   0.0   
50519  0.0   0.0    0.0   0.0   0.0    0.0   0.0   0.0   0.0   0.0  ...   0.0   
50520  0.0   0.0    0.0   0.0   0.0    0.0   0.0   0.0   0.0   0.0  ...   0.0   
50521  0.0   0.0    0.0   0.0   0.0    0.0   0.0   0.0   0.0   0.0  ...   0.0   
50522  0.0   0.0    0.0   0.0   0.0    0.0   0.0   0.0   0.0   0.0  ...   0.0   

       ziarah  zikir  zina  zionis  zodiak  zona  zonasi  zone  zulu  
0         0.0    0.0   0.0     0.0     0.0   0.0     0.0   0.0   0.0  
1         0.0    0.0   0.0     0.0     0.0   0.0     0.0   0.0   0.0  
2         0.0    0.0   0.0     0.0     0.0   0.0     0.0   0.0   0.0  
3         0.0    0.0   0.0     0.0     0.0   0.0     0.0   0.0   0.0  
4         0.0    0.0   0.0     0.0     0.0   0.0     0.0   0.0   0.0  
...       ...    ...   ...     ...     ...   ...     ...   ...   ...  
50518     0.0    0.0   0.0     0.0     0.0   0.0     0.0   0.0   0.0  
50519     0.0    0.0   0.0     0.0     0.0   0.0     0.0   0.0   0.0  
50520     0.0    0.0   0.0     0.0     0.0   0.0     0.0   0.0   0.0  
50521     0.0    0.0   0.0     0.0     0.0   0.0     0.0   0.0   0.0  
50522     0.0    0.0   0.0     0.0     0.0   0.0     0.0   0.0   0.0  

[50523 rows x 8408 columns]

## Feature Expansion (FastText)

In [24]:
import gensim
from gensim.models import FastText

In [32]:

from gensim.models import FastText

# Sesuaikan path ke lokasi file Anda di Google Drive
file_path = 'corpus_indonews_twitter.fasttext'

# Load model FastText
corpus_indonews = FastText.load(file_path)

In [34]:
similar_word_news = corpus_indonews.wv.most_similar("sedih", topn = 10)
similar_word_news

[('kecewa', 0.7592635750770569),
 ('pedih', 0.7580711841583252),
 ('sesal', 0.6346572637557983),
 ('tangis', 0.6192872524261475),
 ('rasa', 0.6064337491989136),
 ('marah', 0.5997285842895508),
 ('bingung', 0.5642061233520508),
 ('kesal', 0.5555573105812073),
 ('senang', 0.5447505116462708),
 ('sumarah', 0.541563093662262)]

In [48]:
def feature_expansion(df, feature):
    for col in df.columns: #loop per kolom
        try:
            sim_word = corpus_indonews.wv.most_similar(col, topn = 10) #mencari Similarity
        except:
            sim_word = []
        if sim_word != []: #kalo similarity-nya tidak kosong
            for term in [sim_word[i][0] for i in range(len(sim_word))]: #loop per-word yang ada di Similarity
                if term in feature:
                    #untuk semua kolom yang mempunyai nilai 0 di kolom, tetapi mempunyai nilai yang bukan 0 pada kolom term
                    #nilainya diganti dengan nilai kolom term yang mempunyai nilai bukan 0
                    condition = (df[col] == 0) & (df[term] != 0)
                    df.loc[condition, col] = df.loc[condition, term]
    return df

In [49]:
feature_ef = tf_idf_vec.get_feature_names_out()
x_uni_fe = feature_expansion(x_uni, feature_ef)

## Modelling

In [50]:
import tensorflow as tf
print(tf.__version__)

2.16.1


In [51]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv1D, Dense, Dropout, LSTM, Input,SpatialDropout1D, GlobalMaxPool1D, MaxPooling1D, Flatten
# Split data
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(x_uni_fe, y_uni, test_size=0.10, random_state=42)
# Reshape data to (samples, timesteps, features)
X_train = np.expand_dims(X_train, axis=1)  # Adding timestep dimension
X_test = np.expand_dims(X_test, axis=1)
# Membuat model hybrid lstm-cnn
def hybrid_lstm_cnn(x_train, y_train, x_test, y_test):


    model = Sequential()
    model.add(Input(shape=(1,x_train.shape[2])))
    model.add(SpatialDropout1D(0.2))
    model.add(LSTM(128, return_sequences=True))
    model.add(Dropout(0.5))
    model.add(Conv1D(filters=128, kernel_size=5, padding='same', activation='relu'))
    model.add(MaxPooling1D(pool_size=2, strides=1, padding='same'))
    model.add(Dropout(0.5))
    model.add(Flatten())
    model.add(Dense(32, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    optimizer = Adam(learning_rate=0.001)
    model.compile(loss='binary_crossentropy',optimizer=optimizer ,metrics=['accuracy'])
    print(model.summary())

    history = model.fit(x_train, y_train, epochs = 10, validation_data=(x_test, y_test), callbacks=[EarlyStopping(monitor='val_loss', patience=3, min_delta=0.0001)] )

    #predict
    predictions = model.predict(x_test)
    print()
    print("Classification Report :")
    print(classification_report(y_test, predictions.round(), digits=6))


    acc_score = accuracy_score(y_test, predictions.round())
    prec_score = precision_score(y_test, predictions.round())
    rec_score = recall_score(y_test, predictions.round())
    f1score = f1_score(y_test, predictions.round())

    return [acc_score,prec_score,rec_score,f1score ]

## Split Data (90:10)

In [52]:
%%time
hasil = []
hasil.append(['1'] + hybrid_lstm_cnn(X_train, Y_train, X_test, Y_test))
hasil.append(['2'] + hybrid_lstm_cnn(X_train, Y_train, X_test, Y_test))
hasil.append(['3'] + hybrid_lstm_cnn(X_train, Y_train, X_test, Y_test))

Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ spatial_dropout1d_4             │ (None, 1, 8408)        │             0 │
│ (SpatialDropout1D)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_4 (LSTM)                   │ (None, 1, 128)         │     4,370,944 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_8 (Dropout)             │ (None, 1, 128)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_4 (Conv1D)               │ (None, 1, 128)         │        82,048 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_4 (MaxPooling1D)  │ (None, 1, 128)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_9 (Dropout)             │ (None, 1, 128)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_4 (Flatten)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 32)             │         4,128 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_9 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,457,153 (17.00 MB)

 Trainable params: 4,457,153 (17.00 MB)

 Non-trainable params: 0 (0.00 B)

...

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ spatial_dropout1d_6             │ (None, 1, 8408)        │             0 │
│ (SpatialDropout1D)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_6 (LSTM)                   │ (None, 1, 128)         │     4,370,944 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_12 (Dropout)            │ (None, 1, 128)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_6 (Conv1D)               │ (None, 1, 128)         │        82,048 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_6 (MaxPooling1D)  │ (None, 1, 128)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_13 (Dropout)            │ (None, 1, 128)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_6 (Flatten)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_12 (Dense)                │ (None, 32)             │         4,128 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_13 (Dense)                │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,457,153 (17.00 MB)

 Trainable params: 4,457,153 (17.00 MB)

 Non-trainable params: 0 (0.00 B)

None
Epoch 1/10
1421/1421 ━━━━━━━━━━━━━━━━━━━━ 54s 36ms/step - accuracy: 0.7066 - loss: 0.5387 - val_accuracy: 0.7500 - val_loss: 0.4581
Epoch 2/10
1421/1421 ━━━━━━━━━━━━━━━━━━━━ 50s 35ms/step - accuracy: 0.7556 - loss: 0.4608 - val_accuracy: 0.7696 - val_loss: 0.4355
Epoch 3/10
1421/1421 ━━━━━━━━━━━━━━━━━━━━ 51s 36ms/step - accuracy: 0.7636 - loss: 0.4490 - val_accuracy: 0.7726 - val_loss: 0.4214
Epoch 4/10
1421/1421 ━━━━━━━━━━━━━━━━━━━━ 51s 36ms/step - accuracy: 0.7679 - loss: 0.4380 - val_accuracy: 0.7776 - val_loss: 0.4181
Epoch 5/10
1421/1421 ━━━━━━━━━━━━━━━━━━━━ 51s 36ms/step - accuracy: 0.7740 - loss: 0.4278 - val_accuracy: 0.7880 - val_loss: 0.4080
Epoch 6/10
1421/1421 ━━━━━━━━━━━━━━━━━━━━ 51s 36ms/step - accuracy: 0.7752 - loss: 0.4217 - val_accuracy: 0.7827 - val_loss: 0.4076
Epoch 7/10
1421/1421 ━━━━━━━━━━━━━━━━━━━━ 52s 37ms/step - accuracy: 0.7806 - loss: 0.4187 - val_accuracy: 0.7970 - val_loss: 0.4029
Epoch 8/10
1421/1421 ━━━━━━━━━━━━━━━━━━━━ 51s 36ms/step - accuracy: 0.7

In [53]:
import os

# Membuat folder 'hasil' jika belum ada
if not os.path.exists('hasil'):
    os.makedirs('hasil')

In [54]:
df = pd.DataFrame(hasil, columns =['Run', 'Accuracy', 'Precision','Recall','F1-Score'])
df.to_excel('hasil/CNN_TF-IDF UniBigram_9010_FastText_Corpus Indonews_Top1.xlsx', index=False)
print('Average Accuracy  :', Average(df['Accuracy'].to_list()))
print('Average Precision :', Average(df['Precision'].to_list()))
print('Average Recall    :', Average(df['Recall'].to_list()))
print('Average F1-Score  :', Average(df['F1-Score'].to_list()))

Average Accuracy  : 0.7931921630714428
Average Precision : 0.7608429685530007
Average Recall    : 0.8459677419354837
Average F1-Score  : 0.8004790199949227
